In [1]:
import geopandas
import pygeos
import pandas as pd
import numpy as np
import momepy
from libpysal.weights import Queen

In [2]:
import tracemalloc

In [3]:
tracemalloc.start()

cross_chunk = pd.read_parquet('../../urbangrammar_samba/spatial_signatures/cross-chunk_indices.pq')

current, peak = tracemalloc.get_traced_memory()
print(f"Current memory usage is {current / 10**6}MB; Peak was {peak / 10**6}MB")

Current memory usage is 1.788752MB; Peak was 1.799134MB


In [4]:
chunk_id = 6

In [5]:
blg = geopandas.read_parquet(f"../../urbangrammar_samba/spatial_signatures/buildings/blg_{chunk_id}.pq")

current, peak = tracemalloc.get_traced_memory()
print(f"Current memory usage is {current / 10**6}MB; Peak was {peak / 10**6}MB")

Current memory usage is 10.22013MB; Peak was 48.570526MB


In [6]:
tess = geopandas.read_parquet(f"../../urbangrammar_samba/spatial_signatures/tessellation/tess_{chunk_id}.pq")

current, peak = tracemalloc.get_traced_memory()
print(f"Current memory usage is {current / 10**6}MB; Peak was {peak / 10**6}MB")

Current memory usage is 18.626027MB; Peak was 310.242884MB


In [7]:
orig_ids = tess.uID.copy()

current, peak = tracemalloc.get_traced_memory()
print(f"Current memory usage is {current / 10**6}MB; Peak was {peak / 10**6}MB")

Current memory usage is 20.759493MB; Peak was 310.242884MB


In [8]:
additional_tess = []
addtional_blg = []
add_tess_data = []
for chunk, inds in cross_chunk.loc[chunk_id].indices.iteritems():
    add_tess = geopandas.read_parquet(f"../../urbangrammar_samba/spatial_signatures/tessellation/tess_{chunk}.pq").iloc[inds]
    additional_tess.append(add_tess)
    add_blg = geopandas.read_parquet(f"../../urbangrammar_samba/spatial_signatures/buildings/blg_{chunk}.pq")
    addtional_blg.append(add_blg[add_blg.uID.isin(add_tess.uID)])
    tess_data = pd.read_parquet(f"../../urbangrammar_samba/spatial_signatures/morphometrics/cells/cells_{chunk}.pq")
    add_tess_data.append(tess_data[tess_data.uID.isin(add_tess.uID)])

current, peak = tracemalloc.get_traced_memory()
print(f"Current memory usage is {current / 10**6}MB; Peak was {peak / 10**6}MB")

Current memory usage is 36.688325MB; Peak was 310.242884MB


In [9]:
add_tess_data.append(pd.read_parquet(f"../../urbangrammar_samba/spatial_signatures/morphometrics/cells/cells_{chunk_id}.pq"))

current, peak = tracemalloc.get_traced_memory()
print(f"Current memory usage is {current / 10**6}MB; Peak was {peak / 10**6}MB")

Current memory usage is 36.692236MB; Peak was 310.242884MB


In [10]:
tess = tess.append(pd.concat(additional_tess, ignore_index=True), ignore_index=True)

current, peak = tracemalloc.get_traced_memory()
print(f"Current memory usage is {current / 10**6}MB; Peak was {peak / 10**6}MB")

Current memory usage is 41.676399MB; Peak was 310.242884MB


In [11]:
blg = blg.append(pd.concat(addtional_blg, ignore_index=True), ignore_index=True)

current, peak = tracemalloc.get_traced_memory()
print(f"Current memory usage is {current / 10**6}MB; Peak was {peak / 10**6}MB")

Current memory usage is 44.613441MB; Peak was 310.242884MB


In [12]:
blg = blg.rename_geometry('buildings')
tess = tess.rename_geometry('tessellation')

current, peak = tracemalloc.get_traced_memory()
print(f"Current memory usage is {current / 10**6}MB; Peak was {peak / 10**6}MB")

Current memory usage is 44.649183MB; Peak was 310.242884MB


In [13]:
df = tess.merge(blg, on='uID', how='left')

current, peak = tracemalloc.get_traced_memory()
print(f"Current memory usage is {current / 10**6}MB; Peak was {peak / 10**6}MB")

Current memory usage is 56.434476MB; Peak was 310.242884MB


In [14]:
w = Queen.from_dataframe(df, geom_col='tessellation')

current, peak = tracemalloc.get_traced_memory()
print(f"Current memory usage is {current / 10**6}MB; Peak was {peak / 10**6}MB")

/opt/conda/lib/python3.7/site-packages/libpysal/weights/weights.py:172: UserWarning: The weights matrix is not fully connected: 
 There are 4 disconnected components.
 There is 1 island with id: 279771.
  warnings.warn(message)


Current memory usage is 271.191743MB; Peak was 5014.933752MB


In [15]:
w3 = momepy.sw_high(k=3, weights=w)

current, peak = tracemalloc.get_traced_memory()
print(f"Current memory usage is {current / 10**6}MB; Peak was {peak / 10**6}MB")

Current memory usage is 558.7676MB; Peak was 5014.933752MB


In [16]:
adj_list = w.to_adjlist(remove_symmetric=True)
adj_list["distance"] = (
    df.buildings.iloc[adj_list.focal]
    .reset_index(drop=True)
    .distance(df.buildings.iloc[adj_list.neighbor].reset_index(drop=True))
)
adj_list = adj_list.set_index(['focal', 'neighbor'])

current, peak = tracemalloc.get_traced_memory()
print(f"Current memory usage is {current / 10**6}MB; Peak was {peak / 10**6}MB")

/opt/conda/lib/python3.7/site-packages/libpysal/weights/weights.py:309: UserWarning: {} islands in this weights matrix. Conversion to an adjacency list will drop these observations!
  "{} islands in this weights matrix. Conversion to an "


Current memory usage is 589.052692MB; Peak was 5014.933752MB


In [17]:
tracemalloc.stop()

In [ ]:
w